<a href="https://colab.research.google.com/github/tomonari-masada/course2023-stats1/blob/main/07_Bayesian_text_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bayesian Text Retrieval

**ランタイムのタイプをGPUにしておく**

## 準備

### インポート

In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

### データセット

In [ ]:
train_corpus, train_labels = fetch_20newsgroups(subset="train", return_X_y=True)
test_corpus, test_labels = fetch_20newsgroups(subset="test", return_X_y=True)
print(f"training size: {len(train_corpus)}\ntest size: {len(test_corpus)}")

### 単語の出現回数を数える

In [ ]:
vectorizer = CountVectorizer(min_df=10, stop_words="english")
X_train = vectorizer.fit_transform(train_corpus).toarray()
X_test = vectorizer.transform(test_corpus).toarray()
vocabulary = vectorizer.get_feature_names_out()
print(f"vocabulary size: {len(vocabulary)}")

## (A) MAP推定（スムージング）

以下は、授業で使ったnotebookからの抜粋。

---
* クエリの尤度を、各文書について求めた単語確率を使って計算する。
  * $n_{q,w}$: クエリ$q$における単語$w$の出現頻度
  * このとき、文書$d$の単語確率を使ったクエリ$q$の対数尤度は、以下の通り。
$$\begin{align}
L_q(d) = \sum_w n_{q,w} \log \phi_{d,w}
\end{align}$$
  * 上の式で、規格化定数の部分は省略している。（ランキングに関係しないため。）
* このように計算されたクエリの尤度によって、検索対象の文書をソートする。
  * $L_q(d)$が大きい順に、文書を検索結果として表示する。
---

* 上の説明にある$L_q(d)$の式の計算を、高速化する。
* $L_q(d)$の式の計算は、単に、内積の計算をしているだけ。
  * クエリ文書の単語の出現回数と・・・
  * 訓練文書の単語確率の対数をとったものとで・・・
  * 内積の計算をしているだけ。
* ということは・・・
* 全テスト文書について、各訓練文書について推定された単語確率で尤度を求めることは、行列の積として書ける。
* 今回は、GPUも使って、高速化する。

* 下のMAP推定は、ディリクレ事前分布を対称なディリクレ分布とし、
* さらにそのパラメータ$\beta_w$をすべて1.01と設定する場合に相当する。

$$
\hat{\phi}_w = \frac{c_w + \beta_w - 1}{\sum_w (c_w + \beta_w - 1)}
$$

In [ ]:
# ディリクレ事前分布のパラメータ
beta = 0.01 + 1.0

X_train = X_train + beta - 1.0
X_train_probs = X_train / X_train.sum(axis=1).reshape(-1, 1)

### クエリの対数尤度を計算するヘルパ関数
* PyTorchを使って、GPU上で計算する。

In [ ]:
import torch

def log_likelihood(x_test, x_train_prob):
  return torch.matmul(
      torch.tensor(x_test, dtype=torch.float32, device="cuda"),
      torch.log(torch.tensor(x_train_prob, dtype=torch.float32, device="cuda")).t()
  ).cpu().numpy()

### 検索の実行

* 全てのtest文書について、その尤度を最大にするtraining文書を求める。

In [ ]:
scores = log_likelihood(X_test, X_train_probs)

In [ ]:
sorted_train_indices = (- scores).argsort(-1)

In [ ]:
top_ranked_train_docs = sorted_train_indices[:,0].reshape(-1)
print(top_ranked_train_docs)

* P@1はprecision at oneの略。
  * https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Precision_at_k

In [ ]:
print(f"P@1={(test_labels == train_labels[top_ranked_train_docs]).sum()/len(test_labels):.3f}")

* 最上位にランキングされた訓練文書がテスト文書と同じカテゴリになっている割合は、0.7ぐらい。

* `beta`をチューニングする。

## (B) ベイズ推測
* 予測分布（＝ディリクレ多項分布）を利用してクエリの予測確率を求める。

* 授業資料より式を下に転載。
  * $\mathbf{x}$のところに検索対象のテキストを代入して、クエリ$\mathbf{x}_0$の予測確率を計算する。
  * $c_{\mathbf{x},w}$は検索対象のテキストにおける単語$w$の出現回数を表す。
  * $c_{\mathbf{x}_0,w}$はクエリにおける単語$w$の出現回数を表す。

$$
p(\mathbf{x}_0|\mathbf{x};\mathbf{\beta})
= \frac{n_0! \Gamma(\sum_{w=1}^W (c_{\mathbf{x},w} + \beta_w))}{\Gamma( \sum_{w=1}^W (c_{\mathbf{x},w} + c_{\mathbf{x}_0,w} + \beta_w) )}
\prod_{w=1}^W
\frac{\Gamma(c_{\mathbf{x},w} + c_{\mathbf{x}_0,w}+\beta_w)}{c_{\mathbf{x}_0,w}!\Gamma(c_{\mathbf{x},w} + \beta_w)}
$$

* 上の式で、$\mathbf{x}_0$がクエリに相当する。
  * よって、$\mathbf{x}_0$だけに依存する項は、検索対象のテキストのランク付けには無関係。
* ディリクレ事前分布は対称ディリクレ分布だと仮定する。
  * つまり、すべての$w$について$\beta_w = \beta$と、同じ値$\beta$を取ると仮定する。

* 以上を踏まえて、テキスト$\mathbf{x}$を使って算出されるクエリ$\mathbf{x}_0$の対数予測確率を書き下す。
  * テキスト$\mathbf{x}$の長さを$l_{\mathbf{x}}$と書くことにする。

$$
\ln p(\mathbf{x}_0|\mathbf{x}_i;\mathbf{\beta})
= \ln \Gamma(l_\mathbf{x} + W\beta) - \ln \Gamma( l_\mathbf{x} + l_{\mathbf{x}_0} + W \beta )
+ \sum_{w=1}^W \big(
\ln \Gamma(c_{\mathbf{x}, w}+c_{\mathbf{x}_0,w}+\beta_w) - \ln \Gamma(c_{\mathbf{x}, w} + \beta_w) \big) + const.
$$

In [ ]:
import torch

X_train_cuda = torch.tensor(X_train, dtype=torch.float32, device="cuda")
train_len = X_train_cuda.sum(-1)
X_test_cuda = torch.tensor(X_test, dtype=torch.float32, device="cuda")
test_len = X_test_cuda.sum(-1)

* 定数の設定

In [ ]:
beta = 0.01 #対称ディリクレ事前分布のパラメータ
vocab_size = X_train.shape[-1]

* $\ln \Gamma(l_\mathbf{x} + W\beta)$を計算する。

In [ ]:
train_lgamma_all = torch.lgamma(train_len + X_train.shape[-1] * beta)

* $l_\mathbf{x} + l_{\mathbf{x}_0}$をブロードキャストで計算する。
  * 検索対象のテキストとテスト用テキスト（クエリとして使用）の
  * すべての組み合わせについて
  * 二つのテキストの長さの和を求める。

In [ ]:
test_train_len = train_len + test_len.unsqueeze(1)

* $\ln \Gamma( l_\mathbf{x} + l_{\mathbf{x}_0} + W \beta )$を計算する。

In [ ]:
lgamma_len = torch.lgamma(test_train_len + vocab_size * beta)

In [ ]:
lgamma_len.shape

* $\ln \Gamma(c_{\mathbf{x}, w} + \beta_w)$を計算する。

In [ ]:
train_lgamma_word = torch.lgamma(X_train_cuda + beta)

* $\ln \Gamma(l_\mathbf{x} + W\beta)
- \sum_{w=1}^W
\ln \Gamma(c_{\mathbf{x}, w} + \beta_w)$を計算する。

In [ ]:
train_lgamma = train_lgamma_all - train_lgamma_word.sum(-1)

In [ ]:
train_lgamma.shape

* $c_{\mathbf{x}, w}+c_{\mathbf{x}_0,w}$をブロードキャストで計算する。

In [ ]:
#X_sum = X_train_cuda + X_test_cuda.unsqueeze(1)

* メモリが溢れてしまうので、ミニバッチ方式で計算することにする。

In [ ]:
import torch

def log_pred_prob(idx1, idx2):
  X_sum = X_train_cuda + X_test_cuda[idx1:idx2].unsqueeze(1) + beta
  log_prob = train_lgamma.reshape(1, -1) - lgamma_len[idx1:idx2]
  log_prob = log_prob + torch.lgamma(X_sum).sum(-1)
  return log_prob

In [ ]:
from tqdm import tqdm

BATCH_SIZE = 4
cnt = 0
for idx in tqdm(range(0, X_test.shape[0], BATCH_SIZE)):
  sorted_train_indices = (- log_pred_prob(idx, idx+BATCH_SIZE)).argsort(-1)
  top_ranked_train_docs = sorted_train_indices[:,0].reshape(-1)
  cnt += (test_labels[idx:idx+BATCH_SIZE] == train_labels[top_ranked_train_docs.cpu()]).sum()

In [ ]:
print(f"P@1={cnt / len(test_labels):.3f}")

* `beta`をチューニングする。